In [ ]:
# 4.0  모듈
# LangChain은 다음과 같은 주요 모듈을 위한 확장 가능한 표준 인터페이스 및 외부 통합 기능을 제공합니다:

# - Model I/O
# 언어 모델과의 인터페이스

# - Retrieval
# 애플리케이션별 데이터를 사용한 인터페이스

# - Agents
# 높은 수준의 지침이 주어지면 체인이 어떤 도구를 사용할지 선택하도록 합니다

# - Chains
# 일반적인 빌딩 블록 구성

# - Memory
# 체인 실행 간 지속적인 응용 프로그램 상태

# - Callbacks
# 모든 체인의 중간 단계 기록 및 스트리밍


In [ ]:
# 4.1 FewShotPromptTemplate

from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
# what is fewshot? fewshot은 기본적으로 모델에게 예제들을 준다는 것과 같다. 더 나은 대답을 할 수 있도록 예제들을 더 줄 수 있다. 
# 구체적으로 대답하는 ai가 필요하다고 가정, 모델에게 어떻게 대답해야하는지에 대한 '예제'를 ai 모델에게 주는 것이 prompt를 사용해서 어떻게 대답해야하는지 알려주는 것보다 좋다! 

from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)
# how to make the PromptTemplate
# template을 만들었을때 좋은 점은 유효성 검사 가능  e.g. country="France" 값이 없으면 에러가 난다. 
# PromptTemplate을 사용하는 또 다른 이유 : 디스트에 저장하고 load할 수 있다. 
# t = PromptTemplate.from_template("What is the capital of {country}")
# t = PromptTemplate(
#   template =  "What is the capital of {country}",
#   input_variables=["country"]
# # 입력변수는 보통 template의 변수가 무엇인지! 
# )
# t.format(country="France")


examples = [
{
"question": "What do you know about France?",
"answer": """
Here is what I know:
Capital: Paris
Language: French
Food: Wine and Cheese
Currency: Euro
""",
},
{
"question": "What do you know about Italy?",
"answer": """
I know this:
Capital: Rome
Language: Italian
Food: Pizza and Pasta
Currency: Euro
""",
},
{
"question": "What do you know about Greece?",
"answer": """
I know this:
Capital: Athens
Language: Greek
Food: Souvlaki and Feta Cheese
Currency: Euro
""",
},
]


# examples를 사용하지 않고, 답변을 요구 했을때  -> 엄청 길게 나열한다. 
# chat.predict("What do you know about France?")


# fewShotPromptTemplate을 사용하기 위해서 첫번째 단계 : 예제의 형식을 지정한다. 
# 예제의 형식 (데이터베이스에서 가져올 수도 있다. )
# example_template = """
#     Human : {question}
#     AI : {answer}
# """
# example_prompt = PromptTemplate.from_template(example_template)
example_prompt = PromptTemplate.from_template("Human:{question}\nAI:{answer}") #위의 문장을 하나로 합쳤다. 


# langchain이 각각의 예제 리스트들을 이 prompt를 사용하여 형식화 한다. 
prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    suffix="Human : What do you know about {country}?",
    # suffix : 형식화된 모든 예제의 마지막에 나오는 내용. e.g. 위의 형식화가 끝나고 사용자의 질문은 무엇인지 나오는거다. 
    input_variables=["country"] #유효성 검사를 위해서 어떤 변수를 suffix에서 사용할건지 지정해줘야한다. 
)

# prompt.format(country="Germany")


chain = prompt| chat
chain.invoke({
    "country" : "Korea"
})


# 1) 해야할 일은 예제들을 가져오고 2) 예제들을 prompt를 사용해서 형식화 해주고 3) 이를 fewShotPromptTemplate에 전달한다. 
#  4) fewShotPromptTemplate은 각각 예제들을 가져와 e.g. example prompt를 이용하여 형식화 해주고 5) 마지막에 질문을 넣어준다. 
